In [6]:
# research/01_verify_coint.ipynb

import sys
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Fix path to import from src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.utils.loader import DataLoader
from src.utils.universe import get_liquid_universe

# --- 1. CONFIG ---
# Pick a pair the Scanner found (or test two known correlated assets)
ASSET_Y = 'BTC' 
ASSET_X = 'ETH'
LOOKBACK_DAYS = 60

# --- 2. GET DATA ---
print("Fetching Data...")
universe = [ASSET_Y, ASSET_X]
loader = DataLoader(universe)
df = loader.fetch_data(LOOKBACK_DAYS)

# Extract Close prices and Log-Transform
closes = df.xs('close', axis=1, level=1).ffill()
log_prices = np.log(closes)

# Split Train/Test (Like the scanner)
train_len = int(len(log_prices) * 0.85)
train = log_prices.iloc[:train_len]
test = log_prices.iloc[train_len:]

# --- 3. CALCULATE HEDGE RATIO (OLS) ---
# Y = beta * X + alpha
x_train = sm.add_constant(train[ASSET_X])
model = sm.OLS(train[ASSET_Y], x_train).fit()
hedge_ratio = model.params[ASSET_X]
intercept = model.params['const']

print(f"Hedge Ratio: {hedge_ratio:.4f}")

# --- 4. CONSTRUCT THE SPREAD ---
# Spread = Y - (beta * X) - alpha
# We construct this for the WHOLE dataset to see how it performs Out-of-Sample
spread = log_prices[ASSET_Y] - (hedge_ratio * log_prices[ASSET_X]) - intercept

# --- 5. VISUALIZATION ---
plt.figure(figsize=(15, 10))

# Plot 1: Normalized Prices (Visual Correlation)
plt.subplot(2, 1, 1)
plt.title(f"Log-Prices: {ASSET_Y} vs {ASSET_X}")
plt.plot(train.index, train[ASSET_Y], label=ASSET_Y)
plt.plot(train.index, train[ASSET_X], label=ASSET_X)
plt.axvline(train.index[-1], color='r', linestyle='--', label='Train/Test Split')
plt.legend()

# Plot 2: The Spread (The most important chart)
plt.subplot(2, 1, 2)
plt.title(f"The Spread (Mean Reversion Check)")
plt.plot(spread.index, spread, color='purple', label='Spread')
plt.axhline(0, color='black', linestyle='--')
plt.axhline(spread.std()*2, color='green', linestyle=':', label='+2 Std Dev')
plt.axhline(spread.std()*-2, color='green', linestyle=':', label='-2 Std Dev')
plt.axvline(train.index[-1], color='r', linestyle='--', label='Out-of-Sample Start')
plt.legend()

plt.tight_layout()
plt.show()

# --- 6. STATS CHECK ---
from statsmodels.tsa.stattools import adfuller
adf = adfuller(spread.iloc[:train_len])
print(f"ADF P-Value (In-Sample): {adf[1]:.5f}")
print(f"Spread Volatility: {spread.std():.4f}")

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import RobustScaler
from src.features.engineering import FeatureEngineer
from src.models.autoencoder import train_autoencoder
from src.utils.universe import get_liquid_universe

# --- INTERNAL HELPER: Doing the heavy lifting ---
def _generate_clustering_results(lookback_days=50):
    print("\n" + "="*50)
    print(f"--- 1. Data Extraction (Lookback: {lookback_days}d) ---")
    universe = get_liquid_universe(50)
    engine = FeatureEngineer(universe)
    engine.load_data(lookback_days=lookback_days) 
    
    print("\n--- 2. Strategic Feature Engineering ---")
    raw_features = engine.calculate_features()
    
    if raw_features.empty:
        print("❌ No features.")
        return pd.DataFrame()

    print("\n--- 3. Training AI Market Mapper ---")
    model, latent_space = train_autoencoder(raw_features)
    print("   Latent Space Dimension: 4 (Rich Feature Extraction)")
    
    # Scale Latent Space
    latent_scaler = RobustScaler()
    latent_scaled = latent_scaler.fit_transform(latent_space)
    
    print("\n--- 4. Clustering Regimes ---")
    n_clusters = 7
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(latent_scaled)
    
    results = raw_features.copy()
    results['cluster'] = clusters
    
    # Sort clusters by Beta for consistent ordering
    cluster_perf = results.groupby('cluster')['beta'].mean()
    sorted_map = {old: new for new, old in enumerate(cluster_perf.sort_values().index)}
    results['cluster'] = results['cluster'].map(sorted_map)
    
    return results

# --- PUBLIC EXPORT: For the Cointegration Scanner ---
def get_cluster_map(lookback_days=60):
    results = _generate_clustering_results(lookback_days)
    if results.empty: return {}
        
    cluster_map = {}
    for c in sorted(results['cluster'].unique()):
        coins = results[results['cluster'] == c].index.tolist()
        cluster_map[c] = coins
        
    return cluster_map

def run_clustering_pipeline():
    print("\n" + "="*50)
    print("--- 1. Data Extraction ---")
    # ... ~30 lines of calculation code ...
    results['cluster'] = results['cluster'].map(sorted_map)
    
    print("\n" + "="*80)
    universe = get_liquid_universe(50)
    engine = FeatureEngineer(universe)
    engine.load_data(lookback_days=50) 
    
    print("\n--- 2. Strategic Feature Engineering ---")
    raw_features = engine.calculate_features()
    
    if raw_features.empty:
        print("❌ No features.")
        return

    print("\n--- 3. Training AI Market Mapper ---")
    # Training the brain to recognize patterns
    model, latent_space = train_autoencoder(raw_features)
    print("   Latent Space Dimension: 4 (Rich Feature Extraction)")
    
    # --- FIX 1: Scale Latent Space ---
    # Even if inputs are scaled, the latent space can have outliers (like STABLE).
    # Scaling here ensures K-Means calculates distances fairly for the main cluster.
    latent_scaler = RobustScaler()
    latent_scaled = latent_scaler.fit_transform(latent_space)
    
    print("\n--- 4. Clustering Regimes ---")
    # --- FIX 2: Increase Clusters to 7 ---
    # 4 buckets was too few. We need buckets for:
    # 1. Stable, 2. Leaders, 3. Memes, 4. Laggards, 5. Defensive Alts, 6. Momentum, 7. Outliers
    n_clusters = 7
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(latent_scaled)
    
    results = raw_features.copy()
    results['cluster'] = clusters
    
    # Sort clusters by Beta (Low Risk -> High Risk) for consistent ordering
    cluster_perf = results.groupby('cluster')['beta'].mean()
    sorted_map = {old: new for new, old in enumerate(cluster_perf.sort_values().index)}
    results['cluster'] = results['cluster'].map(sorted_map)
    
    print("\n" + "="*80)
    print(f"      AI GENERATED MARKET MAP (Sorted by Beta Risk)")
    print("="*80)
    
    for c in sorted(results['cluster'].unique()):
        cluster_df = results[results['cluster'] == c]
        coins = cluster_df.index.tolist()
        avg = cluster_df.mean()
        
       # --- STRICT HIERARCHICAL LABELING (Refined V3) ---
        
        label = "UNKNOWN"
        desc = "General Market Movement"
        
        # 1. SPECIAL CASES: Defensive
        if avg['beta'] < 0.5:
            label = "🛡️ DEFENSIVE / HEDGE"
            desc = "Uncorrelated or Safe Haven (Stablecoins/Gold)"
            
        # 2. THE TRAP: Toxic Volatility (High Risk, Bad Returns)
        # CRITICAL FIX: Check this BEFORE 'High Octane' to catch falling knives like Cluster 3
        elif (avg['volatility_z'] > 1.2 or avg['beta'] > 1.3) and avg['alpha'] < 0:
            label = "⚠️ TOXIC VOLATILITY / DOWNTREND"
            desc = "High Risk but Negative Alpha (Avoid/Short)"

        # 3. THE WINNERS: High Octane (High Risk, Good Returns)
        # Must have positive Alpha to earn the "High Octane" badge
        elif (avg['volatility_z'] > 1.2 or avg['beta'] > 1.3) and avg['alpha'] > 0:
            label = "🔥 HIGH OCTANE / MEMES"
            desc = "Aggressive Growth Leaders (High Beta Winners)"
            
        # 4. TRUE LEADERS: Significant Outperformance
        # Raised Alpha threshold from 0.10 -> 0.15 to exclude "The Blob" (Cluster 1)
        elif avg['alpha'] > 0.15:
            label = "🚀 TRUE MARKET LEADERS"
            desc = "Strongest Trenders beating the Index significantly"
            
        # 5. THE LAGGARDS: Bleeding Value
        elif avg['alpha'] < -0.05:
            label = "💤 LAGGARDS / WEAK"
            desc = "Underperforming the benchmark consistently"
            
        # 6. MOMENTUM: Volume/RSI Anomalies
        elif abs(avg['rsi_rel']) > 15 or abs(avg['volume_flow']) > 1.0:
            label = "⚡ MOMENTUM PLAY"
            desc = "High relative volume or RSI divergence"
            
        # 7. THE BLOB: Broad Market
        # If it didn't pass the high bar for 'Leader' (>0.15), it falls here.
        # This captures BTC/ETH and the 35 coins moving with the tide.
        else:
            label = "🌊 BROAD MARKET / SYSTEMATIC"
            desc = "Moving in lockstep with the Index (Beta ~1.0)"

        print(f"\nCLUSTER {c} : {label} ({len(coins)} coins)")
        print(f"   {desc}")
        print(f"Coins: {coins}")
        
        # Stats Table
        print(f"   Beta: {avg['beta']:.2f} | Alpha: {avg['alpha']:.4f} | Vol Z: {avg['volatility_z']:.2f}")
        print(f"   Flow: {avg['volume_flow']:.2f} | RSI Rel: {avg['rsi_rel']:.1f} | Funding: {avg['funding_sentiment']:.4f}")
        print("-" * 80)

if __name__ == "__main__":
    run_clustering_pipeline()